# ETL Pipeline

## Extract Data

In [1]:
# %pip install -q tweepy
# %pip install -q python-decouple

In [2]:
# import necessay packages
import pandas as pd
import tweepy as tw
# from decouple import config

In [3]:
# config
consumer_key = 'ALS2SK37YuglyLYSyGGpfrbmY'
consumer_secret = '2Vyr3pBaHDsNaVX9IslmRHq25xR4zJjQjHpjUFTZzXj74rkvwS'
access_token = '1521183878576230400-4DzxTK7h5tI1gQ1f4QwoEWqfn0FNuV'
access_token_secret = 'sft4cbN41jMtVneJLD794rYRGJraHbb9ug4YELbZHFSyV'
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAA6ScAEAAAAAnv1iKJAyg2INl1nA7%2BB0z%2FJbpK4%3DbrVth6B429DvCW0vsS4ufO6pLzmsAnhyYpaPRfWGoMWBKc6uRM'
user_id = '90676816'

In [4]:
# connect API
permission = tw.OAuthHandler(consumer_key, consumer_secret) # this object of ours will load the authorization request
permission.set_access_token(access_token, access_token_secret) # get ownership of our access object
client = tw.Client(bearer_token=bearer_token)
api = tw.API(permission) # use the API class, native to the library to generate the connection

In [5]:
query = 'Atletico-MG -filter:retweets OR Atlético Mineiro -filter:retweets OR #Galo -filter:retweets OR #ClubeAtléticoMineiro -filter:retweets'
galo_tweets = api.search_tweets(q=query, tweet_mode='extended', count=100)

id = []
created_at = []
full_text = []
favorite_count = []
retweet_count = []

for twitters in galo_tweets:
    id.append(twitters.id)
    created_at.append(twitters.created_at)
    full_text.append(twitters.full_text)
    favorite_count.append(twitters.favorite_count)
    retweet_count.append(twitters.retweet_count)

    # create dataframe
    df_final = {
      'tweet_id': id, 'created_at':created_at, 'text':full_text, 'likes':favorite_count, 
      'retweets':retweet_count
    }

In [6]:
search_tweets_df = pd.DataFrame(df_final)
search_tweets_df

,tweet_id,created_at,text,likes,retweets
0,1617357461312135168,2023-01-23 03:04:02+00:00,Primeira rodada do Campeonato Mineiro vem se d...,0,0
1,1617268268267446275,2023-01-22 21:09:37+00:00,"“Estamos atentos ao mercado, conversando diari...",444,18
2,1617258837840646145,2023-01-22 20:32:08+00:00,Atlético-MG promove leilão de camisas utilizad...,0,0
3,1617258827573006338,2023-01-22 20:32:06+00:00,Atlético-MG já apresenta uma ideia de jogo cla...,1,0
4,1617258817863180288,2023-01-22 20:32:04+00:00,Hulk acredita que o Atlético-MG vai ficar mais...,0,0
...,...,...,...,...,...
95,1616692519923761154,2023-01-21 07:01:48+00:00,Mineiro: Ronaldo remonta Cruzeiro para desafia...,1,0
96,1616681043326652416,2023-01-21 06:16:11+00:00,Hoje tem a estreia do Campeonato Mineiro; \n\n...,0,0
97,1616641141281112064,2023-01-21 03:37:38+00:00,"Vá ao estádio, cante os 90 minutos e empurre o...",1,2
98,1616614720387317760,2023-01-21 01:52:39+00:00,@realfutebolnews Flamengo: Petkovic\nFluminens...,0,0


In [7]:
# empty lists are created that will store the data extracted from the API call
id = []
date = []
text = []
retweets = []
answers = []
likes = []
mentions = []

# API call script with loop and pagination
for tweet in tw.Paginator(
    client.get_users_tweets, id=user_id, exclude=['retweets', 'replies'], #start_time = '2023-01-01T00:00:00Z',
    tweet_fields=['id', 'text', 'created_at', 'public_metrics'], max_results=100).flatten(limit=3200):

    # inserting the results in the lists
    id.append(tweet['id'])
    date.append(tweet['created_at'])
    text.append(tweet['text'])
    retweets.append(tweet['public_metrics']['retweet_count'])
    answers.append(tweet['public_metrics']['reply_count'])
    likes.append(tweet['public_metrics']['like_count'])
    mentions.append(tweet['public_metrics']['quote_count'])

    # create dataframe
    results = {
      'tweet_id':id, 'date':date, 'text':text, 'likes':likes, 
      'retweets':retweets, 'answers':answers, 'mentions':mentions
    }

In [8]:
paginator_df = pd.DataFrame(results)
paginator_df

,tweet_id,date,text,likes,retweets,answers,mentions
0,1617221417413607425,2023-01-22 18:03:27+00:00,"No primeiro desafio do ano, Hulk e Everson mos...",1280,85,14,5
1,1617205278973255680,2023-01-22 16:59:19+00:00,Hulk 🫶🏾 Gol 🫶🏾 #Galo 🐔 https://t.co/Qswonqmgs8,3003,199,27,11
2,1617185352334073856,2023-01-22 15:40:08+00:00,🤔 Vocês sabiam que o Allan já marcou um gol pe...,1005,63,29,21
3,1617168108489244680,2023-01-22 14:31:37+00:00,Ontem o Allan completou 3 anos no Atlético e h...,691,31,6,2
4,1617150018997075968,2023-01-22 13:19:44+00:00,"No dia em que completou três anos de #Galo, Al...",3683,191,78,35
...,...,...,...,...,...,...,...
820,1597674255432433664,2022-11-29 19:30:00+00:00,"🇧🇷🐔 Na Copa de 70, o Brasil encantou o mundo c...",626,43,1,0
821,1597655379998973952,2022-11-29 18:15:00+00:00,The invisible wall! 💪🛡️ @arsenal\n\n#FIFAWorld...,2874,160,34,15
822,1597647047871451136,2022-11-29 17:41:53+00:00,"🤙🏾🐔 Feliz aniversário, @fcbarcelona_br! A magi...",3317,338,49,36
823,1597638740490752000,2022-11-29 17:08:53+00:00,Sene𝗚𝗔𝗟𝗢 classificadaço! 🐔🤝🇸🇳\n\n#FIFAWorldCup,1135,56,33,7


In [10]:
test = pd.read_parquet('atletico_own_tweets.parquet')
test

,tweet_id,date,text,likes,retweets,answers,mentions
0,1617168108489244680,2023-01-22 14:31:37+00:00,Ontem o Allan completou 3 anos no Atlético e h...,205,8,7,0
1,1617150018997075968,2023-01-22 13:19:44+00:00,"No dia em que completou três anos de #Galo, Al...",1713,99,50,22
2,1616979898114969600,2023-01-22 02:03:44+00:00,https://t.co/8MJnAxfHUh,636,20,3,1
3,1616979889424592898,2023-01-22 02:03:42+00:00,https://t.co/qUpaWOMfHF,669,21,3,2
4,1616979880683487232,2023-01-22 02:03:40+00:00,Tarde de estreias no Horto. Que a nova caminha...,1387,72,17,2
...,...,...,...,...,...,...,...
820,1597638740490752000,2022-11-29 17:08:53+00:00,Sene𝗚𝗔𝗟𝗢 classificadaço! 🐔🤝🇸🇳\n\n#FIFAWorldCup,1136,56,33,7
821,1597621258841370624,2022-11-29 15:59:25+00:00,#CoudetNoGalo 🏴🏳️,527,35,4,2
822,1597621255678545923,2022-11-29 15:59:24+00:00,Chacho foi conhecer o seu novo escritório hoje...,4100,243,91,96
823,1597605790344675330,2022-11-29 14:57:57+00:00,🇳🇱🐔 Você disse Laranja Mecânica em campo? 🤔 \n...,1662,109,12,8
